# 演示使用向量来检索。

## 1 生成测试数据集

In [2]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

In [3]:
from openai import OpenAI

client = OpenAI()

def generate_data_by_prompt(prompt):
    data = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=2048,
        temperature=0.5,
        top_p = 1
    )
    return data.choices[0].text

prompt = """请你生成50条淘宝京东里的商品的标题，每条在30个字左右，品类是3C数码产品，标题里往往也会有一些促销类的信息，每行一条。"""
data = generate_data_by_prompt(prompt)
print(data)



1.【限时抢购】华为P30 Pro全面屏智能手机
2.【限量发售】苹果AirPods Pro降噪耳机
3.【新品上市】小米9T Pro游戏手机
4.【超值特惠】华硕ROG游戏笔记本电脑
5.【限时优惠】索尼WH-1000XM3无线耳机
6.【新品预售】三星Galaxy Fold折叠屏手机
7.【限量抢购】罗技G502游戏鼠标
8.【清仓特价】华为Mate 20 Pro智能手机
9.【疯狂秒杀】小米电视4A 65英寸
10.【超值推荐】苹果iPad Air 2019平板电脑
11.【限时特惠】华硕RT-AX88U路由器
12.【新品抢先】联想Yoga C930轻薄本
13.【限量发售】索尼PlayStation 5游戏机
14.【限时折扣】华为MateBook 13笔记本
15.【新品上架】小米手环4运动手表
16.【超值优惠】苹果iPhone XR智能手机
17.【狂欢节】三星QLED电视65英寸
18.【限时秒杀】戴尔Alienware游戏台式机
19.【新品抢购】华硕ZenFone 6旗舰手机
20.【清仓特价】索尼PS4 Pro游戏机
21.【限量发售】华为平板M6 10.8英寸
22.【限时折扣】小米电视4S 55英寸
23.【超值推荐】苹果MacBook Pro 2019款
24.【新品预售】联想智能手表E2
25.【限量抢购】罗技G915无线游戏键盘
26.【限时优惠】华硕VivoBook S14轻薄本
27.【疯狂秒杀】三星Galaxy S10+智能手机
28.【超值特惠】索尼Alpha A7R IV相机
29.【新品上市】小米路由器AX3600
30.【限时特惠】苹果AirPods 2无线耳机
31.【限量发售】华为Mate 30 Pro 5G手机
32.【新品抢先】联想Yoga S940轻薄本
33.【限时折扣】索尼Xperia 1智能手机
34.【清仓特价】华硕ROG游戏台式机
35.【超值推荐】苹果iPad Mini 2019平板电脑
36.【限时秒杀】小米电视4C 50英寸
37.【新品抢购】华为畅享10 Plus智能手机
38.【限量发售】索尼XB900N降噪耳机
39.【限时优惠】华硕VivoBook 14轻薄本
40.【疯狂秒杀】三星QLED电视55英寸
41.【超值特惠】罗技MX Master 3无线鼠标
42.【新品上市】小米手

In [4]:
import pandas as pd

product_names = data.strip().split('\n')
df = pd.DataFrame({'product_name': product_names})
df.head()

,product_name
0,1.【限时抢购】华为P30 Pro全面屏智能手机
1,2.【限量发售】苹果AirPods Pro降噪耳机
2,3.【新品上市】小米9T Pro游戏手机
3,4.【超值特惠】华硕ROG游戏笔记本电脑
4,5.【限时优惠】索尼WH-1000XM3无线耳机


In [5]:
# 去掉序号
df.product_name = df.product_name.apply(lambda x: x.split('.')[1].strip())
df.head()

,product_name
0,【限时抢购】华为P30 Pro全面屏智能手机
1,【限量发售】苹果AirPods Pro降噪耳机
2,【新品上市】小米9T Pro游戏手机
3,【超值特惠】华硕ROG游戏笔记本电脑
4,【限时优惠】索尼WH-1000XM3无线耳机


In [6]:
# 如法炮制，再生成一些女装的商品名称，覆盖不同的品类，这样后面我们演示搜索效果的时候就会方便一点。
clothes_prompt = """请你生成50条淘宝网里的商品的标题，每条在30个字左右，品类是女性的服饰箱包等等，标题里往往也会有一些促销类的信息，每行一条。"""
clothes_data = generate_data_by_prompt(clothes_prompt)
clothes_product_names = clothes_data.strip().split('\n')
clothes_df = pd.DataFrame({'product_name': clothes_product_names})
clothes_df.product_name = clothes_df.product_name.apply(lambda x: x.split('.')[1].strip())
clothes_df.head()

,product_name
0,夏日必备！清新花朵印花连衣裙
1,轻松出游，时尚百搭小包包
2,优雅女神范！蕾丝拼接修身连衣裙
3,限时折扣！百搭防晒草帽
4,专业运动服饰，让你更自信


In [7]:
df = pd.concat([df, clothes_df], axis=0)
df = df.reset_index(drop=True)
display(df)

,product_name
0,【限时抢购】华为P30 Pro全面屏智能手机
1,【限量发售】苹果AirPods Pro降噪耳机
2,【新品上市】小米9T Pro游戏手机
3,【超值特惠】华硕ROG游戏笔记本电脑
4,【限时优惠】索尼WH-1000XM3无线耳机
...,...
95,甜蜜小公主！蝴蝶结装饰连衣裙
96,轻松休闲，时尚牛仔短裤
97,气质女神必备！简约百搭手提包
98,优雅女神范！蕾丝拼接修身连衣裙


In [8]:
# 将DataFrame保存到CSV文件
def save_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"DataFrame saved to '{filename}'")

save_to_csv(df, 'data/data.csv')

DataFrame saved to 'data/data.csv'


In [2]:
import pandas as pd

# 从CSV文件加载DataFrame，如果文件不存在则返回None
def load_from_csv(filename):
    try:
        df = pd.read_csv(filename)
        print(f"DataFrame loaded from '{filename}'")
        return df
    except FileNotFoundError:
        print(f"File '{filename}' not found")
        return None


df = load_from_csv('data/data.csv')

DataFrame loaded from 'data/data.csv'


## 2 将数据转换为embedding后进行存入csv

In [6]:
from openai import OpenAI

client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"): 
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

#df['ada_embedding'] = df.product_name.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
#df.to_csv('data/data_em.csv', index=False)

In [3]:
df = load_from_csv('data/data_em.csv')

DataFrame loaded from 'data/data_em.csv'


In [4]:
print(df.iloc[3]['ada_embedding'])

[-0.01552498061209917, -0.015754736959934235, -0.02538263238966465, 0.0018571991240605712, 0.028052184730768204, -0.054616425186395645, -0.02132359892129898, 0.09496605396270752, -0.018872862681746483, -0.010377339087426662, -0.021181369200348854, 0.024573015049099922, -0.07325953245162964, 0.012209922075271606, 0.031728290021419525, 0.013391527347266674, -0.09514111280441284, 0.013172711245715618, -0.01378539577126503, 0.011914520524442196, 0.012286507524549961, 0.04010893777012825, 0.05107160657644272, 0.0073905037716031075, -0.019376138225197792, -0.003205651417374611, -0.015809440985322, -0.028533579781651497, 0.03652035817503929, 0.01091343816369772, 0.004105531610548496, -0.042778488248586655, 0.0507652647793293, -0.03328188136219978, -0.04603884369134903, 0.017483381554484367, 0.01874157413840294, -0.01734115183353424, -0.001582311699166894, 0.01792101375758648, 0.006154194474220276, 0.027658317238092422, 0.018282059580087662, 0.03599520027637482, 0.011597237549722195, 0.0383365

## 3 使用余弦相似度进行向量搜索

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
# 定义计算余弦相似度的函数
import numpy as np


# 将字符串形式的嵌入向量转换为列表
def str_to_list(s):
    # 去除字符串两端的括号，然后按逗号分割，并转换为浮点数
    return [float(x.strip()) for x in s.strip('[]').split(',')]


# 计算余弦相似度
def cos_sim(embedding1, embedding2):
    # 将嵌入向量转换为数组形式
    embedding1_array = np.array(embedding1).reshape(1, -1)
    embedding2_array = np.array(embedding2).reshape(1, -1)
    
    # 计算余弦相似度
    similarity = cosine_similarity(embedding1_array, embedding2_array)
    return similarity[0][0]


def recommend_product(df, product_name, n=3, pprint=True):
    product_embedding = get_embedding(product_name)
    df["similarity"] = df.ada_embedding.apply(lambda x: cos_sim(str_to_list(x), product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .product_name
    )
    if pprint:
        for r in results:
            print(r)
    return results

results = recommend_product(df, "智能大屏手机", n=3)

【限时抢购】华为P30 Pro全面屏智能手机
【疯狂秒杀】三星Galaxy S10+智能手机
【清仓特价】华为Mate 20 Pro智能手机
